In [1]:
import requests
from bs4 import BeautifulSoup
import requests
from paperview.retrieval.biorxiv_api import (
    query_content_detail_by_doi, 
    query_content_detail_by_interval,
    get_all_content_details_by_interval,
    query_article_by_doi,
    ArticleDetail
)
from paperview.retrieval.biorxiv_web import (
    get_content_detail_for_page,
    NamedTemporaryPDF
    )

from pikepdf import Pdf, PdfImage
from io import StringIO

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
from io import StringIO
from pdfminer.high_level import extract_text_to_fp
from pdfminer.layout import LAParams

%load_ext autoreload
%autoreload 2


In [49]:
url = 'https://www.biorxiv.org/content/10.1101/2022.07.21.500746v1'
content_detail = get_content_detail_for_page(url=url)

In [47]:


with NamedTemporaryPDF(content_detail.pdf_url) as f:
    doc = Pdf.open(f)
    images = []
    for ii, page in enumerate(doc.pages):
        for jj, (name, raw_image) in enumerate(page.images.items()):
            image = PdfImage(raw_image)
            images.append(image)
    
    output_string = StringIO()
    with open(f, 'rb') as in_file:
        parser = PDFParser(in_file)
        doc = PDFDocument(parser)
        rsrcmgr = PDFResourceManager()
        device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for page in PDFPage.create_pages(doc):
            interpreter.process_page(page)
            
        text = output_string.getvalue()
        
    layout_output_string = StringIO()
    with open(f, 'rb') as fin:
        extract_text_to_fp(fin, 
                           layout_output_string, 
                           laparams=LAParams(), 
                           output_type='html', codec=None)
        
        layout_text = layout_output_string.getvalue()

In [64]:
content_detail.title

'Evaluating instruments for assessing healthspan: a multi-center cross-sectional study on health-related quality of life (HRQL) and frailty in the companion dog'

In [ ]:
content_detail.abstract

In [ ]:
print(text)

In [ ]:
print(layout_text)

In [ ]:
from IPython.core.display import display, HTML
display(HTML(layout_text))